## Plotting testing

In [13]:
import pickle
  
# Open the file in binary mode
with open('/home/k2/Work/SCU/eVision/evision-prediction-python/response.pkl', 'rb') as file:
      
    # Call load method to deserialze
    response = pickle.load(file)
  
response.keys()

Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-02-27 13:18:58           64
config.json                                    2023-02-27 13:18:58         1931
variables.h5                                   2023-02-27 13:18:58      5189848
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dropout
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


dict_keys(['history', 'predictions', 'actual_data', 'confidence_interval'])

In [14]:
response['actual_data']

array([ 52671.,  55027.,  62539.,  79184., 113921.,  93323.,  69158.,
        48685.,  34720.,  25809.,  23719.,  23930.,  23620.,  25789.,
        28073.,  31342.,  32433.,  34811.,  38128.,  40953.,  42072.,
        44980.,  47787.,  53380.,  54062.,  53992.,  51879.,  45240.,
        40250.,  37830.,  34570.,  34995.,  33011.,  32613.,  32040.,
        29647.,  28923.,  31572.,  35788.,  37529.,  38334.,  39760.,
        44666.,  46154.,  57259.,  63472.,  75736., 109460., 139349.,
       140147., 137393., 165087., 185776., 167327., 151793., 130083.,
       112565.,  92119.,  67757.,  57788.,  58208.,  59247.,  61996.,
        57705.])

In [52]:
import pandas as pd

ci = response.get('confidence_interval')
df = pd.DataFrame({'actual_data': response.get('actual_data'), 'predictions': response.get('predictions')})
df['week'] = range(1, len(df)+1)
df['predictions_upper'] = df['predictions'] + df['predictions']* 0.01 * ci
df['predictions_lower'] = df['predictions'] - df['predictions']* 0.01 * ci


# num_weeks = [range(1, len(response.get('predictions'))+1)]
df
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(
    go.Line(
        name='Actual Data',
        x=df['week'],
        y=df['actual_data']
    )
)


fig.add_trace(
    go.Line(
        name='Predictions',
        x=df['week'],
        y=df['predictions'],

    )
)

fig.add_trace(
    go.Scatter(
        name='Upper Bound',
        x=df['week'],
        y=df['predictions_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter(
        name='Lower Bound',
        x=df['week'],
        y=df['predictions_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 0, 0, 0.2)',
        fill='tonexty',
        showlegend=False
    )
)

fig.update_layout(
    xaxis={'title': "Number of weeks"},
    yaxis= {'title': "ILI Cases", 'tickformat':'.2e'},
    title="<b>Influenza</b> Predictions",
    title_x=0.5,
)

fig.show()

/home/k2/Work/SCU/eVision/evision-prediction-python/.venv/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




# FULL SCRATCHPAD

In [57]:
batch_size = 200

train_test_split = 0.75
trainX, testX = X[:round(X.shape[0]*train_test_split)].to_numpy(), X[round(X.shape[0]*train_test_split):].to_numpy() 
trainy, testy = y[:round(y.shape[0]*train_test_split)].to_numpy(), y[round(y.shape[0]*train_test_split):].to_numpy()
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
trainy = np.reshape(trainy, (trainy.shape[0], trainy.shape[1], 1))
testy = np.reshape(testy, (testy.shape[0], testy.shape[1], 1))

model1 = Sequential()
model1.add(LSTM(327, input_shape=(trainX.shape[1], trainX.shape[2])))
model1.add(Dropout(rate=0.1))
model1.add(Dense(1))

model1.compile(loss='mse', optimizer='adam')

response = {}

history = model1.fit(trainX, trainy, batch_size=batch_size, epochs=5, shuffle=False)
response['history'] = history
pred = model1.predict(testX)
pred_unnorm = (pred * std[0]) + mean[0]
testy_unnorm = (testy * std[0]) + mean[0]

response['predictions'] = pred_unnorm.reshape(testy.shape[0])
response['actual_data'] = testy_unnorm.reshape(testy.shape[0])
response['confidence_interval'] = ci

Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.7916
Epoch 2/5
1/1 [==============================] - 0s 50ms/step - loss: 0.7363
Epoch 3/5
1/1 [==============================] - 0s 74ms/step - loss: 0.6760
Epoch 4/5
1/1 [==============================] - 0s 69ms/step - loss: 0.6327
Epoch 5/5
2/2 [==============================] - 1s 12ms/step


IndexError: invalid index to scalar variable.